A notebook for calculating the Shannon Entropy of my multiple sequence alignments. Adapted from Joe Healey's Shannon.py.

In [37]:
from Bio import AlignIO
import project_pipeline.scripts.utils as utils
import pandas as pd
import numpy as np
import sys
import os

In [38]:
__author__ = "Joe R. J. Healey"
__version__ = "1.0.0"
__title__ = "ShannonMSA"
__license__ = "GPLv3"
__author_email__ = "J.R.J.Healey@warwick.ac.uk"

def parseMSA(msa, alnformat, verbose):
    """Parse in the MSA file using Biopython's AlignIO"""

    alignment = AlignIO.read(msa, alnformat)

    # Do a little sanity checking:
    seq_lengths_list = []
    n_seqs = len(alignment._records)
    for record in alignment:
       seq_lengths_list.append(len(record))

    seq_lengths = set(seq_lengths_list)

    if verbose > 0: print("Alignment length is:" + str(list(seq_lengths)))

    if len(seq_lengths) != 1:
        sys.stderr.write("Your alignment lengths aren't equal. Check your alignment file.")
        sys.exit(1)

    index = range(1, list(seq_lengths)[0]+1)

    return alignment, list(seq_lengths), index, n_seqs

##################################################################
# Function to calcuate the Shannon's entropy per alignment column
# H=-\sum_{i=1}^{M} P_i\,log_2\,P_i (http://imed.med.ucm.es/Tools/svs_help.html)
# Gaps and N's are included in the calculation
##################################################################

def shannon_entropy(list_input):
    """Calculate Shannon's Entropy per column of the alignment (H=-\sum_{i=1}^{M} P_i\,log_2\,P_i)"""

    import math
    unique_base = set(list_input)
    M   =  len(list_input)
    entropy_list = []
    # Number of residues in column
    for base in unique_base:
        n_i = list_input.count(base) # Number of residues of type i
        P_i = n_i/float(M) # n_i(Number of residues of type i) / M(Number of residues in column)
        entropy_i = P_i*(math.log(P_i,2))
        entropy_list.append(entropy_i)

    sh_entropy = -(sum(entropy_list))

    return sh_entropy


def shannon_entropy_list_msa(alignment):
    """Calculate Shannon Entropy across the whole MSA"""

    shannon_entropy_list = []
    for col_no in range(len(list(alignment[0]))):
        list_input = list(alignment[:, col_no])
        shannon_entropy_list.append(shannon_entropy(list_input))

    return shannon_entropy_list

def plot(index, sel, verbose):
    """"Create a quick plot via matplotlib to visualise"""
    import matplotlib.pyplot as plt

    if verbose > 0: print("Plotting data...")

    plt.plot(index, sel)
    plt.xlabel('MSA Position Index', fontsize=16)
    plt.ylabel('Shannon Entropy', fontsize=16)

    plt.show()

def running_mean(l, N):
    sum = 0
    result = list(0 for x in l)

    for i in range( 0, N ):
        sum = sum + l[i]
        result[i] = sum / (i+1)

    for i in range( N, len(l) ):
        sum = sum - l[i-N] + l[i]
        result[i] = sum / N

    return result

def region_shannon(array):
    '''
    Determine average shannon entropy and percentage of high-variable columns
    for a given sequence array
    '''

    reg_mean = np.mean(array)
    reg_n_high = len(array[array >= 2.0])
    reg_n = len(array)
    perc_n_high = reg_n_high / reg_n

    return [reg_n, reg_n_high, perc_n_high, reg_mean]


def compute_shannon(msa, reg1, reg2, alnformat='fasta', verbose=0):

    alignment, seq_lengths, index, nseq = parseMSA(msa, alnformat, verbose)
    sel = np.array(shannon_entropy_list_msa(alignment))

    prefixes = ['w_', 'reg1_', 'reg2_', 'both_reg_', 'o_reg_']
    suffixes = ['n', 'nh', 'perc_h', 'mean']
    keys = ['nseq'] # Start with number of sequences
    for p in prefixes:
        for s in suffixes:
            keys.append(p + s)

    values = [nseq] # Start with number of sequences

    # Statistics for entire array
    values = values + region_shannon(sel)

    # Statistics for regions 1 and 2
    # One of my proteins has the incorrect upper bound, so we will have to make an exception for that
    try:
        values = values + region_shannon(np.take(sel, reg1))
    except IndexError:
        reg1 = reg1[:-1]
        values = values + region_shannon(np.take(sel, reg1))
    values = values + region_shannon(np.take(sel, reg2))

    # Statistics for both regions
    values = values + region_shannon(np.take(sel, reg1 + reg2))

    # Statistics for all other regions
    sub_array = np.delete(sel, reg1 + reg2)
    values = values + region_shannon(sub_array)

    temp_dic = {k: v for k, v in zip(keys, values)}

    return temp_dic


def main(df, path):
    '''
    Collect Shannon entropy info for each msa in our dataset. For a summary statistic, we will count the number
    of indices with H >= 2.0. We will also collect information on how many sequences there are.
    '''
    shan_df = pd.DataFrame()

    for idx, row in df.iterrows():

        unp = row['uniprot']
        clst = row['cluster']
        reg1 = utils.string2range(row['region_1'])
        reg2 = utils.string2range(row['region_2'])

        # Adjust regions to serve as indices
        reg1 = [x - 1 for x in reg1]
        reg2 = [x - 1 for x in reg2]

        # Open the cluster file to get the UniRef IDs
        fn = f'{unp}_{clst}.a3m'
        fp = os.path.join(path, unp, fn) # File path looks like data/O08967/O08967_000.a3m
        temp_dic = compute_shannon(fp, reg1, reg2)

        temp_df = pd.DataFrame([{'uniprot': unp, 'cluster': clst, **temp_dic}])

        shan_df = pd.concat([shan_df, temp_df]).reset_index(drop=True)

    return shan_df


In [39]:
# Autoinhibitory
df = pd.read_csv('./project_pipeline/data/ai_pdb_clusters.tsv', sep='\t')[['uniprot', 'cluster', 'region_1', 'region_2']].drop_duplicates().reset_index(drop=True)
path = './autoinhibition_clusters/'

shan_df = main(df, path)
shan_df.head()

,uniprot,cluster,nseq,w_n,w_nh,w_perc_h,w_mean,reg1_n,reg1_nh,reg1_perc_h,...,reg2_perc_h,reg2_mean,both_reg_n,both_reg_nh,both_reg_perc_h,both_reg_mean,o_reg_n,o_reg_nh,o_reg_perc_h,o_reg_mean
0,P07038,U10-003,11,920,392,0.426087,1.629037,26,0,0.0,...,0.546875,1.882610,90,35,0.388889,1.465711,830,357,0.430120,1.646747
1,P07038,000,4,920,0,0.000000,0.093473,26,0,0.0,...,0.000000,0.012676,90,0,0.000000,0.117185,830,0,0.000000,0.090902
2,P07038,U10-007,11,920,359,0.390217,1.583648,26,0,0.0,...,0.500000,1.834187,90,32,0.355556,1.581026,830,327,0.393976,1.583932
3,P07038,U10-008,11,920,432,0.469565,1.705492,26,0,0.0,...,0.640625,2.000250,90,41,0.455556,1.600153,830,391,0.471084,1.716914
4,P07038,U100-005,101,920,528,0.573913,2.091565,26,0,0.0,...,0.703125,2.542571,90,45,0.500000,2.020354,830,483,0.581928,2.099287


In [40]:
# Two-domain
md = pd.read_csv('./project_pipeline/data/md_pdb_clusters.tsv', sep='\t')[['uniprot', 'cluster', 'region_1', 'region_2']].drop_duplicates().reset_index(drop=True)
path2 = './multi_domain_clusters'

shan_md = main(md, path2)
shan_md.head()

,uniprot,cluster,nseq,w_n,w_nh,w_perc_h,w_mean,reg1_n,reg1_nh,reg1_perc_h,...,reg2_perc_h,reg2_mean,both_reg_n,both_reg_nh,both_reg_perc_h,both_reg_mean,o_reg_n,o_reg_nh,o_reg_perc_h,o_reg_mean
0,D9N168,U10-007,11,579,238,0.411054,1.615098,167,95,0.568862,...,0.331731,1.401645,375,164,0.437333,1.686246,204,74,0.362745,1.484311
1,D9N168,U100-004,101,579,342,0.590674,2.132564,167,130,0.778443,...,0.480769,1.856641,375,230,0.613333,2.191116,204,112,0.549020,2.024932
2,D9N168,U10-005,11,579,263,0.454231,1.648029,167,106,0.634731,...,0.350962,1.447240,375,179,0.477333,1.707169,204,84,0.411765,1.539315
3,D9N168,U100-002,101,579,364,0.628670,2.243077,167,139,0.832335,...,0.504808,1.939129,375,244,0.650667,2.315004,204,120,0.588235,2.110859
4,D9N168,U10-009,11,579,264,0.455959,1.684303,167,112,0.670659,...,0.360577,1.434128,375,187,0.498667,1.751471,204,77,0.377451,1.560832


In [41]:
shan_df.to_csv('./project_pipeline/data/ai_af2_clusters_shannon_entropy.csv', index=False)

In [42]:
shan_md.to_csv('./project_pipeline/data/md_af2_clusters_shannon_entropy.csv', index=False)